In [104]:
def multiply_truncate(a: int, b: int) -> int:
    # Ensure inputs are treated as signed 8-bit values
    # if a < -128 or a > 127 or b < -128 or b > 127:
    #     raise ValueError("Inputs must be signed 8-bit integers (-128 to 127).")

    # Perform multiplication (result fits in signed 16-bit range)
    product = a * b
    width=16
    mask = (1 << width) - 1
    # format(product & mask, f'0{width}b')
    print(f"Full Value of c: {format(product & mask, f'0{width}b')}, decimal: {product} ")
    
    # Mask to 16-bit signed range
    product &= 0xFFFF
    if product & 0x8000:  # if negative in 16-bit two's complement
        product -= 0x10000

    # Extract the top 8 bits (bits 15 down to 8)
    top8 = (product >> 8) & 0xFF

    return top8

def twocomp_c(c):
    # # Mask to 8 bits
    c &= 0xFF
    # # Format as 8-bit binary string
    bin_str = f"{c:08b}"
    val = 1
    final = 0

    # Process bits 7 down to 1 (excluding MSB)
    for i in range(len(bin_str) - 1, 0, -1):
        final += val * int(bin_str[i])
        val *= 2

    # Handle MSB (sign bit)
    final += -val * int(bin_str[0])

    return final

def add_shifted_top8(c: int, d: int) -> int:
    # Ensure inputs are signed 8-bit values
    # if not (-128 <= c <= 127 and -128 <= d <= 127):
    #     raise ValueError("Inputs must be signed 8-bit integers (-128 to 127).")

    # Shift both values left by 8 bits
    c_shifted = c << 8
    d_shifted = d << 8

    width=16
    mask = (1 << width) - 1
    # format(product & mask, f'0{width}b')
    print(f"Full Value of c: {format(c_shifted & mask, f'0{width}b')}, decimal: {c_shifted} ")
    print(f"Full Value of d: {format(d_shifted & mask, f'0{width}b')}, decimal: {d_shifted} ")

    # Add them together
    e = c_shifted + d_shifted
    width=17
    mask = (1 << width) - 1
    # format(product & mask, f'0{width}b')
    print(f"Full Value of e: {format(e & mask, f'0{width}b')}, decimal: {e} ")

    # Mask to 17-bit range
    e &= 0x1FFFF
    if e & 0x10000:  # if negative in 17-bit two's complement
        e -= 0x20000

    # Extract the top 8 bits (bits 15 down to 8)
    top8 = (e >> 9) & 0xFF

    # Interpret as signed 8-bit
    if top8 & 0x80:
        top8 -= 0x100

    return top8


def compute_f(e_top8: int, b: int) -> int:
    """
    e_top8: the top 8 bits of e (signed 8-bit value)
    b: signed 8-bit value (-128 to 127)
    Returns f: the signed 8-bit value of the top 8 bits after e - (256*b)
    """

    # Ensure inputs are in signed 8-bit range
    # if not (-128 <= e_top8 <= 127 and -128 <= b <= 127):
    #     raise ValueError("Inputs must be signed 8-bit integers (-128 to 127).")

    # Step 1: Reconstruct e into 17-bit space by shifting top8 back left by 9
    e = e_top8 << 9
    width=17
    mask = (1 << width) - 1
    # format(product & mask, f'0{width}b')
    print(f"Full Value of e: \t{format(e & mask, f'0{width}b')}, decimal: {e} ")

    # Step 2: Compute e - (256 * b)
    e = e - (256 * b)
    width=17
    mask = (1 << width) - 1
    print(f"Full Value of 256*b: \t{format(256*b & mask, f'0{width}b')} ({256*b})");
    print(f"Full Value of -(256*b): {format(-(256*b) & mask, f'0{width}b')} ({-(256*b)})");

    width=18
    mask = (1 << width) - 1
    print(f"Full Value of f: \t{format(e & mask, f'0{width}b')} ({e})");
    
    # Step 3: Mask to 18-bit range
    e &= 0x3FFFF                # keep only 18 bits (2^18 - 1 = 0x3FFFF)
    if e & 0x20000:             # if sign bit (bit 17) is set in 18-bit two's complement
        e -= 0x40000            # subtract 2^18 to sign-extend into Python int
    
    # Step 4: Extract the new top 8 bits (bits 17 down to 10)
    f = (e >> 10) & 0xFF        # shift right so bits 17..10 become bits 7..0
    
    # Step 5: Interpret as signed 8-bit
    if f & 0x80:                # if sign bit of 8-bit result is set
        f -= 0x100              # convert to negative Python int

    return f

In [115]:
# Test vectors 13–40
a_values = [
    -128,  # 13: 0x80
    -1,    # 14: 0xFF
    64,    # 15: 0x40
    96,    # 16: 0x60
    80,    # 17: 0x50
    -112,  # 18: 0x90
    127,   # 19: 0x7F
    127,   # 20: 0x7F
    -128,  # 21: 0x80
    -128,  # 22: 0x80
    1,     # 23: 0x01
    2,     # 24: 0x02
    3,     # 25: 0x03
    16,    # 26: 0x10
    1,     # 27: 0x01
    85,    # 28: 0x55
    -86,   # 29: 0xAA
    85,    # 30: 0x55
    -86,   # 31: 0xAA
    1,     # 32: 0x01
    127,   # 33: 0x7F
    -128,  # 34: 0x80
    -1,    # 35: 0xFF
    19,    # 36: 0x13
    107,   # 37: 0x6B
    -13,   # 38: 0xF3
    32,    # 39: 0x20
    16     # 40: 0x10
]

b_values = [
    -1,    # 13: 0xFF
    -128,  # 14: 0x80
    64,    # 15: 0x40
    96,    # 16: 0x60
    -48,   # 17: 0xD0
    -112,  # 18: 0x90
    2,     # 19: 0x02
    127,   # 20: 0x7F
    2,     # 21: 0x02
    -128,  # 22: 0x80
    -16,   # 23: 0xF0
    -16,   # 24: 0xF0
    -16,   # 25: 0xF0
    15,    # 26: 0x0F
    -128,  # 27: 0x80
    85,    # 28: 0x55
    -86,   # 29: 0xAA
    -86,   # 30: 0xAA
    85,    # 31: 0x55
    -128,  # 32: 0x80
    1,     # 33: 0x01
    1,     # 34: 0x01
    1,     # 35: 0x01
    -25,   # 36: 0xE7
    -110,  # 37: 0x92
    12,    # 38: 0x0C
    4,     # 39: 0x04
    -16    # 40: 0xF0
]

d_values = [
    127,   # 13: 0x7F
    127,   # 14: 0x7F
    0,     # 15: 0x00
    0,     # 16: 0x00
    0,     # 17: 0x00
    0,     # 18: 0x00
    127,   # 19: 0x7F
    127,   # 20: 0x7F
    -128,  # 21: 0x80
    -128,  # 22: 0x80
    16,    # 23: 0x10
    0,     # 24: 0x00
    -1,    # 25: 0xFF
    -128,  # 26: 0x80
    127,   # 27: 0x7F
    85,    # 28: 0x55
    -86,   # 29: 0xAA
    85,    # 30: 0x55
    -86,   # 31: 0xAA
    127,   # 32: 0x7F
    -128,  # 33: 0x80
    127,   # 34: 0x7F
    -1,    # 35: 0xFF
    -92,   # 36: 0xA4
    16,    # 37: 0x10
    126,   # 38: 0x7E
    -128,  # 39: 0x80
    15     # 40: 0x0F
]


In [118]:
# a = -1
# b = 127
# d = -128

for i in range(11, len(a_values)):
    print(f"\n\nTest Vector {i+13}")

    a = a_values[i]
    b = b_values[i]
    d = d_values[i]
    
    width=8
    mask = (1 << width) - 1
    print(f"Value of a: \t{format(a & mask, f'0{width}b')} ({a}, {hex(a)})");
    print(f"Value of b: \t{format(b & mask, f'0{width}b')} ({b}, {hex(b)})");
    print(f"Value of d: \t{format(d & mask, f'0{width}b')} ({d}, {hex(d)})\n");
    
    print(f"Equation 1: a * b = c")
    c = multiply_truncate(a, b)
    
    c_decimal = twocomp_c(c)
    print(f"Top 8 bits of c: {c:08b}, decimal: {c_decimal} ")
    
    print(f"\nEquation 2: e = c + d")
    e = add_shifted_top8(c_decimal, d)
    e_decimal = twocomp_c(e)
    print(f"Top 8 bits of e: {format(e & mask, f'0{width}b')}, decimal: {e_decimal} ")
    
    print(f"\nEquation 3: f = e - (256 * b)")
    f = compute_f(e, b)
    f_decimal = twocomp_c(f)
    
    print(f"Top 8 bits of f: \t{format(f & mask, f'0{width}b')}, decimal: {f_decimal}, hex: {hex(f)} ")

# Clock Cycles 25, 155, 275, 395, 515, 635, 755, 875, 995, 1115, 1235



Test Vector 24
Value of a: 	00000010 (2, 0x2)
Value of b: 	11110000 (-16, -0x10)
Value of d: 	00000000 (0, 0x0)

Equation 1: a * b = c
Full Value of c: 1111111111100000, decimal: -32 
Top 8 bits of c: 11111111, decimal: -1 

Equation 2: e = c + d
Full Value of c: 1111111100000000, decimal: -256 
Full Value of d: 0000000000000000, decimal: 0 
Full Value of e: 11111111100000000, decimal: -256 
Top 8 bits of e: 11111111, decimal: -1 

Equation 3: f = e - (256 * b)
Full Value of e: 	11111111000000000, decimal: -512 
Full Value of 256*b: 	11111000000000000 (-4096)
Full Value of -(256*b): 00001000000000000 (4096)
Full Value of f: 	000000111000000000 (3584)
Top 8 bits of f: 	00000011, decimal: 3, hex: 0x3 


Test Vector 25
Value of a: 	00000011 (3, 0x3)
Value of b: 	11110000 (-16, -0x10)
Value of d: 	11111111 (-1, -0x1)

Equation 1: a * b = c
Full Value of c: 1111111111010000, decimal: -48 
Top 8 bits of c: 11111111, decimal: -1 

Equation 2: e = c + d
Full Value of c: 1111111100000000, dec